## Junos PyEZ Configuration
PyEZ provides a *Config* class which simplifies the process of loading and committing configuration changes to a Junos device. In addition, PyEZ integrates with the Jinja2 templating engine to simplify the process of creating the actual configuration snippet and offers utilities for comparing configurations, rolling back configuration changes, and locking or unlocking the configuration database.

In [1]:
from jnpr.junos import Device

vsrx1 = Device(host='vsrx1')
vsrx1.open()

Device(vsrx1)

Configuration requires that the *jnpr.junos.utils.config* module 
* create a configuration instance variable using the **Config()** class;
* lock the configuration database using the **lock()** method;
* a configuration file is loaded into the device’s candidate configuration by passing the *template_path* and *template_vars* argument to the **load()** method; 
* check the configuration with the **commit_check()** method; 
* print the diff with the **diff()** method;
* commit the configuration with the **commit()** method; and
* finally unlock the configuration database with the  **unlock()** method

In [2]:
from jnpr.junos.utils.config import Config

conf = Config(vsrx1)

In [3]:
try:
    conf.lock()
except LockError as err:
    print "Failed exclusive lock on config database: {0}".format(err)
    print "Disconnecting from device"
    vsrx1.close()

    seanw@Seans-iMac:~/Documents/jupyter_notebook/junos_pyez$ more hostname.set 
    set system host-name {{ host_name }}

In [4]:
try:
    conf.load(template_path='hostname.set', template_vars={'host_name': 'foo-bar'})
except ConfigLoadError as err:
    print "Configuration load error: {0}".format(err)
    print "Performing rollback 0"
    conf.rollback(rb_id=0)
    print "Unlocking configuration"
    conf.unlock()
    print "Disconnecting from device"
    vsrx1.close()

The **pdiff()** methods can take an unnamed optional argument that is an integer representing the rollback ID. 

In [5]:
conf.pdiff()


[edit system]
-  host-name vsrx1;
+  host-name foo-bar;



In [6]:
try:
    conf.commit_check()
except ConfigLoadError as err:
    print "Configuration load error: {0}".format(err)
    print "Performing rollback 0"
    conf.rollback(rb_id=0)
    print "Unlocking configuration"
    conf.unlock()
    print "Disconnecting from device"
    vsrx1.close() 

The **commit()** method takes several optional arguments including *confirm* where the value is an integer specifying the number of minutes before a rollback occurs. A commit must be invoked with the **commit()** method before the timer expires otherwise a rollback occurs.

In [7]:
conf.commit(comment='Conf by PyEZ')

True

In [8]:
conf.unlock()

True

Likewise Junos rollbacks can be performed to rollback the configuration with the **rollback()** method, which takes the *rb_id* argument with an integer value the specifies what configuration to rollback to (1 being the last commit).

In [9]:
conf.rollback(rb_id=1)

True

In [10]:
conf.pdiff()


[edit system]
-  host-name foo-bar;
+  host-name vsrx1;



In [11]:
conf.commit(comment='Rollback conf by PyEZ')

True

In [12]:
vsrx1.close()

    seanw@vsrx1> show system commit                      
    0   2017-02-04 13:45:55 UTC by seanw via netconf
        Rollback conf by PyEZ
    1   2017-02-04 13:45:15 UTC by seanw via netconf
        Conf by PyEZ
    [...snipped...]    

### Jinja2 Templating
Jinja2 is used to generate documents (configurations) based on templates and uses the .j2 file extension. Variables are marked in the template with the syntax: **{{ var_name }}**. Jinja2 also supports some control structures (if statements and for loops). These structures use a **{% ... %}** syntax.

    seanw@Seans-iMac:~/jupyter_notebook/junos_pyez$ more interface_template.j2 
    {%- if interfaces %}
    interfaces {
        {%- for interface in interfaces %}
        {{ interface.name }} {
            {%- if interface.sub_interfaces %}
            flexible-vlan-tagging;
            encapsulation flexible-ethernet-services;
            {%- for sub_interface in interface.sub_interfaces %}
            unit {{ sub_interface.vlan }} {
                vlan-id {{ sub_interface.vlan }};
                family inet {
                    address {{ sub_interface.ip_addr }};
                }
            }
            {%- endfor %}
            {%- else %}
            unit 0 {
                {%- if interface.ip_addr %}
                family inet {
                    address {{ interface.ip_addr }};
                }
                {%- else %}
                family ethernet-switching {
                    interface-mode {{ interface.mode }};
                    vlan {
                        members [{%- for vlan in interface.vlans %} {{ vlan }} {%- endfor %}]
                    }
                }
                {%- endif %}
            }
            {%- endif %}
        }
        {%- endfor %}
    }
    {% endif %}

### YAML (Template Variables)
YAML stands for "Yaml Ain't Markup Language" and is human-readable language with less markup than XML and is a superset of JSON. YAML is used for “user to program” communication as it allows users to provide data. It is designed to translate to data structures which are common to various languages (cross language: Python, Perl, Ruby, etc). In Junos configuration it is used to define variable values that will be rendered with a Jinja2 template.

    Seans-iMac:~/jupyter_notebook/junos_pyez$ more interface_template.j2 
    ---
    interfaces:
    - name: ge-0/0/0
      sub_interfaces:
      - vlan: 161
        ip_addr: 172.16.1.1/24
      - vlan: 162
        ip_addr: 172.16.2.1/24
    - name: ge-0/0/1
      ip_addr: 9.9.9.1/24
    - name: ge-0/0/2
      mode: trunk
      vlans:
      - 500
      - 600-650
      - 700
    - name: ge-0/0/3
      sub_interfaces:
      - { vlan: 101, ip_addr: 10.10.1.254/24 }
      - { vlan: 201, ip_addr: 10.20.1.254/24 }
      - { vlan: 301, ip_addr: 10.30.1.254/24 }

### Render Junos Configuration in Python
A configuration file can be loaded into a Junos device’s candidate configuration by passing the template_path and template_vars argument to the **load()** method. However, with Python the configuration can be rendered and the resulting output presented as well.

In [13]:
from jinja2 import Template
from glob import glob
import yaml

# YAML file template variables (input):
with open(glob('*.yml')[0]) as my_vars:
    tvars = yaml.load(my_vars.read())

# Jinja2 template file:
with open(glob('*.j2')[0]) as my_template:
    template_format = my_template.read()

# Create a instance of the Template class    
template = Template(template_format)

# Render the configuration using the render() method
print template.render(tvars)


interfaces {
    ge-0/0/0 {
        flexible-vlan-tagging;
        encapsulation flexible-ethernet-services;
        unit 161 {
            vlan-id 161;
            family inet {
                address 172.16.1.1/24;
            }
        }
        unit 162 {
            vlan-id 162;
            family inet {
                address 172.16.2.1/24;
            }
        }
    }
    ge-0/0/1 {
        unit 0 {
            family inet {
                address 9.9.9.1/24;
            }
        }
    }
    ge-0/0/2 {
        unit 0 {
            family ethernet-switching {
                interface-mode trunk;
                vlan {
                    members [ 500 600-650 700]
                }
            }
        }
    }
    ge-0/0/3 {
        flexible-vlan-tagging;
        encapsulation flexible-ethernet-services;
        unit 101 {
            vlan-id 101;
            family inet {
                address 10.10.1.254/24;
            }
        }
        unit 201 {
            vlan-